# SQL and Python Integration Demo

This notebook demonstrates how to:

1. Connect to an SQLite database using Python.
2. Execute basic SQL queries (SELECT, INSERT, UPDATE, DELETE).
3. Use `pandas` to read SQL query results into a DataFrame.
4. Write DataFrames back to the database.
5. Perform simple ETL (Extract, Transform, Load) operations.
6. Visualize query results.

Make sure you have:
- `sqlite3` (part of the standard library, so no extra install needed)
- `pandas` (install via `pip install pandas`)
- `matplotlib` (install via `pip install matplotlib`)

A sample `company_data.db` file with `Employees` and `Departments` tables is assumed. If you don’t have one yet, run the provided SQL commands to create the necessary tables.


In [6]:
# If running in a notebook, it's good practice to have plots inline
%matplotlib inline
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
## 1. Connecting to a SQLite Database

We will connect to an SQLite database file named `company_data.db`.
If it doesn't exist, it will be created when we connect. But for demonstration,
we assume you've created it using DB Browser for SQLite.


In [8]:
# Connect to the database (create connection object)
conn = sqlite3.connect('company_data.db')

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Check existing tables (if any)
tables = cursor.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
print("Current tables:", tables)


Current tables: []


In [ ]:
If `Employees` and `Departments` tables aren’t present, let’s create them and insert some sample data.
If they already exist, this step can be skipped or commented out.


In [11]:
# Create Employees table if it doesn't exist
cursor.execute("""
CREATE TABLE IF NOT EXISTS Employees (
    ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Name TEXT,
    Age INTEGER,
    Position TEXT,
    DeptID INTEGER
);
""")

# Create Departments table if it doesn't exist
cursor.execute("""
CREATE TABLE IF NOT EXISTS Departments (
    ID INTEGER PRIMARY KEY AUTOINCREMENT,
    DepartmentName TEXT
);
""")

conn.commit()

# Insert sample departments
cursor.execute("INSERT INTO Departments (DepartmentName) VALUES (?)", ("Human Resources",))
cursor.execute("INSERT INTO Departments (DepartmentName) VALUES (?)", ("Engineering",))
cursor.execute("INSERT INTO Departments (DepartmentName) VALUES (?)", ("Data Science",))

# Insert sample employees
cursor.execute("INSERT INTO Employees (Name, Age, Position, DeptID) VALUES (?,?,?,?)", ("Alice Smith", 30, "Data Scientist", 3))
cursor.execute("INSERT INTO Employees (Name, Age, Position, DeptID) VALUES (?,?,?,?)", ("Bob Johnson", 45, "Manager", 1))
cursor.execute("INSERT INTO Employees (Name, Age, Position, DeptID) VALUES (?,?,?,?)", ("Carol Davis", 29, "Data Analyst", 3))

conn.commit()


In [10]:
# Check existing tables (if any)
tables = cursor.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
print("Current tables:", tables)

Current tables: [('Employees',), ('sqlite_sequence',), ('Departments',)]


In [12]:
# Insert data into Employees table
cursor.execute("INSERT INTO Employees (Name, Age, Position) VALUES ('Alice Smith', 30, 'Data Scientist')")
cursor.execute("INSERT INTO Employees (Name, Age, Position) VALUES ('Bob Johnson', 45, 'Manager')")
cursor.execute("INSERT INTO Employees (Name, Age, Position) VALUES ('Carol Davis', 29, 'Data Analyst')")
conn.commit()

# Let's verify that the rows have been inserted by selecting them
cursor.execute("SELECT * FROM Employees")
rows = cursor.fetchall()
rows



[(1, 'Alice Smith', 30, 'Data Scientist', 3),
 (2, 'Bob Johnson', 45, 'Manager', 1),
 (3, 'Carol Davis', 29, 'Data Analyst', 3),
 (4, 'Alice Smith', 30, 'Data Scientist', 3),
 (5, 'Bob Johnson', 45, 'Manager', 1),
 (6, 'Carol Davis', 29, 'Data Analyst', 3),
 (7, 'Alice Smith', 30, 'Data Scientist', None),
 (8, 'Bob Johnson', 45, 'Manager', None),
 (9, 'Carol Davis', 29, 'Data Analyst', None)]

In [15]:
cursor.execute("DELETE FROM Employees WHERE ROWID NOT IN (SELECT MIN(ROWID) FROM Employees GROUP BY Name);")
conn.commit()


In [16]:
# Verify the result
cursor.execute("SELECT * FROM Employees;")
for row in cursor.fetchall():
    print(row)

(1, 'Alice Smith', 30, 'Data Scientist', 3)
(2, 'Bob Johnson', 45, 'Manager', 1)
(3, 'Carol Davis', 29, 'Data Analyst', 3)


In [ ]:
## 3. SELECT and WHERE

The `SELECT` statement is used to retrieve data from a table. The `WHERE` clause allows us to filter rows based on conditions.


In [17]:
# SELECT all columns from Employees
cursor.execute("SELECT * FROM Employees;")
print("All Employees:")
for row in cursor.fetchall():
    print(row)

All Employees:
(1, 'Alice Smith', 30, 'Data Scientist', 3)
(2, 'Bob Johnson', 45, 'Manager', 1)
(3, 'Carol Davis', 29, 'Data Analyst', 3)


In [18]:
# SELECT specific columns
cursor.execute("SELECT Name, Position FROM Employees;")
print("\nJust Name and Position:")
for row in cursor.fetchall():
    print(row)



Just Name and Position:
('Alice Smith', 'Data Scientist')
('Bob Johnson', 'Manager')
('Carol Davis', 'Data Analyst')


In [20]:
cursor.execute("SELECT Name, Age FROM Employees WHERE Age > 30;")
print("\nEmployees older than 30:")
for row in cursor.fetchall():
    print(row)


Employees older than 30:
('Alice Smith', 30)
('Bob Johnson', 45)


In [ ]:
## 4. ORDER BY and LIMIT

`ORDER BY` allows us to sort the result-set by one or more columns.
`LIMIT` restricts how many rows are returned.


In [22]:
cursor.execute("SELECT Name, Age FROM Employees ORDER BY Age ASC;")
print("Employees sorted by Age (descending):")
for row in cursor.fetchall():
    print(row)

Employees sorted by Age (descending):
('Carol Davis', 29)
('Alice Smith', 30)
('Bob Johnson', 45)


In [ ]:
## 5. UPDATE and DELETE

`UPDATE` modifies existing rows in the table.
`DELETE` removes rows from the table.

**Use these commands with caution in real-world scenarios!**


In [23]:
# UPDATE: Increase Carol's age by 1
cursor.execute("UPDATE Employees SET Age = 30 WHERE Name = 'Carol Davis';")
conn.commit()

In [24]:
# Check the result
cursor.execute("SELECT Name, Age FROM Employees WHERE Name='Carol Davis';")
print("Carol after update:", cursor.fetchone())


Carol after update: ('Carol Davis', 30)


In [25]:
# DELETE: Remove Bob Johnson
cursor.execute("DELETE FROM Employees WHERE Name='Bob Johnson';")
conn.commit()
# Check the remaining employees
cursor.execute("SELECT * FROM Employees;")
print("Employees after deleting Bob:")
for row in cursor.fetchall():
    print(row)

Employees after deleting Bob:
(1, 'Alice Smith', 30, 'Data Scientist', 3)
(3, 'Carol Davis', 30, 'Data Analyst', 3)


In [ ]:
## 6. Using pandas to View and Manipulate Data

We can use `pandas` to read SQL query results directly into a DataFrame for easier viewing and further analysis.


In [29]:
# Load the Employees table into a pandas DataFrame
employees_df = pd.read_sql_query("SELECT * FROM Employees;", conn)
print("Employees DataFrame:")
employees_df

Employees DataFrame:


,ID,Name,Age,Position,DeptID
0,1,Alice Smith,30,Data Scientist,3
1,3,Carol Davis,30,Data Analyst,3


In [30]:
# Load the Departments table into a pandas DataFrame
departments_df = pd.read_sql_query("SELECT * FROM Departments;", conn)
print("\nDepartments DataFrame:")
departments_df



Departments DataFrame:


,ID,DepartmentName
0,1,Human Resources
1,2,Engineering
2,3,Data Science
3,4,Human Resources
4,5,Engineering
5,6,Data Science


In [ ]:
## 3. Performing Joins and Aggregations

We can use SQL to perform joins and aggregations and retrieve the results as pandas DataFrames.


In [31]:
# Join Employees with Departments
query = """
SELECT Employees.Name, Employees.Age, Employees.Position, Departments.DepartmentName
FROM Employees
JOIN Departments ON Employees.DeptID = Departments.ID
"""

joined_df = pd.read_sql_query(query, conn)
print("Joined Employees and Departments:")
joined_df

Joined Employees and Departments:


,Name,Age,Position,DepartmentName
0,Alice Smith,30,Data Scientist,Data Science
1,Carol Davis,30,Data Analyst,Data Science


In [32]:
# Aggregation: Count employees in each department
query = """
SELECT d.DepartmentName, COUNT(e.ID) AS NumEmployees
FROM Employees e
JOIN Departments d ON e.DeptID = d.ID
GROUP BY d.DepartmentName
"""
aggregation_df = pd.read_sql_query(query, conn)
print("\nNumber of Employees per Department:")
aggregation_df


Number of Employees per Department:


,DepartmentName,NumEmployees
0,Data Science,2


In [ ]:
## 4. Manipulating Data in pandas

Once the data is in a pandas DataFrame, we can use pandas methods for further analysis and visualization.


In [ ]:
# Example: Add a new column for age categories
employees_df['AgeCategory'] = pd.cut(employees_df['Age'], bins=[0, 30, 40, 100], labels=["Young", "Mid", "Senior"])
print("Employees with Age Category:")
print(employees_df)

# Example: Filter employees older than 30
older_employees = employees_df[employees_df['Age'] > 30]
print("\nEmployees Older Than 30:")
print(older_employees)
